# 2.0 Coordinates to Postcode
Convert the longitude and latitude in the Zoopla data to postcodes. Unfortunately we don't have the postcode or house number in the Zoopla data. So we will infer this from the property longitude and latitude. Datasets of all postcodes for a given area, including their mean latitude and longitude are available from Source: https://www.doogal.co.uk/AdministrativeAreas, which we refer to as the mapping file. We will then use the inferred property postcode to join other data (e.g. deprivation data, flood risk data, EPC data (averages for that postcode and road) and government historic property prices (averages for that postcode, house type, year, etc).

In [1]:
from collections import Counter
import os
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', 100)

### Read in Zoopla data and mapping files that convert co-ordinates to postcodes
Also concatenate the Nuneaton and Hinckley files

In [2]:
DATA_FOLDER = os.path.join('data', 'raw')
SAVE_FOLDER = os.path.join('data', 'processed')

In [3]:
zoopla_df_filename_nuneaton = 'zoopla_properties_nuneaton_180123.csv'
zoopla_df_filename_hinckley = 'zoopla_properties_hinckley_180123.csv'

zoopla_df_nuneaton = pd.read_csv(os.path.join(DATA_FOLDER, zoopla_df_filename_nuneaton))
zoopla_df_hinckley = pd.read_csv(os.path.join(DATA_FOLDER, zoopla_df_filename_hinckley))
zoopla_df = zoopla_df_nuneaton.append(zoopla_df_hinckley).drop_duplicates()

mapping_df_filename_nuneaton = 'Nuneaton and Bedworth postcodes.csv'
mapping_df_filename_hinckley = 'Hinckley and Bosworth postcodes.csv'

mapping_df_nuneaton = pd.read_csv(os.path.join(DATA_FOLDER, mapping_df_filename_nuneaton))
mapping_df_hinckley = pd.read_csv(os.path.join(DATA_FOLDER, mapping_df_filename_hinckley))
mapping_df = mapping_df_nuneaton.append(mapping_df_hinckley).drop_duplicates()

In [4]:
print(mapping_df.shape)
display(mapping_df.head())

(6886, 17)


,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,Ward,Parish,Introduced,Terminated,Altitude,Country,Last Updated,Quality,LSOA Code,LSOA Name
0,CV10 0AA,Yes,52.526750,-1.460760,436681,292234,SP366922,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,NaN,85,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
1,CV10 0AB,Yes,52.527391,-1.459293,436780,292306,SP367923,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,NaN,86,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
2,CV10 0AD,Yes,52.527600,-1.461965,436599,292328,SP365923,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,NaN,86,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
3,CV10 0AE,No,52.526840,-1.462793,436543,292243,SP365922,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,2009-11-01,86,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
4,CV10 0AF,Yes,52.538271,-1.467776,436196,293512,SP361935,Weddington,"Nuneaton and Bedworth, unparished area",2006-04-01,NaN,84,England,2022-11-25,Within the building of the matched address clo...,E01031113,Nuneaton and Bedworth 001D


### Map latitude and longitude to postcode

Create function that takes the longitude and latitude from the Zoopla data and find the closest one in the mapping data, returning the corresponding postcode in the mapping file.

In [5]:
def get_closest_postcode(latitude, longitude, map_df):
    
    """
    Find the nearest longitude and latitude in mapping file and get the postcode.
    This uses the Euclidean (rather than Haversine) distance, since the distances will be small
    such that the Earth's curvature need not be considered.
    An alternative could be to use geopy.reverse() to convert coordinates to postcodes.
    
    Parameters
    ----------
    - latitude (float): latitude of the Zoopla property
    - longitude (float): longitude of the Zoopla property
    - map_df (pandas.DataFrame): dataframe that maps coordinates to postcodes
    
    Returns
    -------
    - postcode (string): postcode closely mapping the Zoopla property
    """
    
    # squared euclidean distance between Zoopla property and each mapping dataset postcode mean
    # this uses array broadcasting in numpy
    sq_distances = (np.array(latitude) - np.array(map_df['Latitude']))**2 + \
        (np.array(longitude) - np.array(map_df['Longitude']))**2 
    
    min_sq_distance_row = np.argmin(sq_distances)
    
    return map_df.iloc[min_sq_distance_row]['Postcode']
    

In [6]:
zoopla_df['postcode_test_1'] = zoopla_df[['latitude', 'longitude']].apply(lambda x: get_closest_postcode(x[0], x[1], mapping_df), axis=1)

In [7]:
zoopla_df[['latitude', 'longitude', 'postcode_test_1']].head(10)

,latitude,longitude,postcode_test_1
0,52.520170,-1.455287,CV11 4FS
1,52.520170,-1.455287,CV11 4FS
2,52.520170,-1.455287,CV11 4FS
3,52.520170,-1.455287,CV11 4FS
4,52.543770,-1.463799,CV10 0FH
5,52.519066,-1.495866,CV10 7RU
6,52.541720,-1.433190,CV11 7AB
7,52.525670,-1.534454,CV10 9NG
8,52.518400,-1.478219,CV11 5XJ
9,52.513065,-1.493124,CV10 7HA


This works OK but sometimes we get an adjacent postcode to the one we want. Instead, let's try combining it with the road name by loading the EPC data which has postcodes and names, and joining to the Zoopla data on the street name.

### Try getting postcode from road name using EPC data
Data available from https://epc.opendatacommunities.org/domestic/search  
Concatenate the Nuneaton and Hinckley EPC data

In [8]:
def get_most_common_postcode(postcode_list):
    
    """
    Return the most frequent value in a list
    """
    
    if not postcode_list:
        return None
    else:
        postcode_counter = Counter(postcode_list)
        return postcode_counter.most_common()[0][0]
    

In [9]:
def get_street_name(address_1, address_2):
    
    """
    Get street name from first two street address fields
    """
    
    street_and_road = re.compile(r'^\d,\s+')
    
    # if street name starts with a number (maybe followed by comma) and a space, likely next part is street name
    if street_and_road.match(address_1):
        street = re.split(street_and_road, address_1)[1].lower()
        
    # otherwise choose the second part of the address as the street name
    else:
        street = str(address_2).lower()
        
    return street


In [10]:
def get_postcode(latitude, longitude, street_name, method='closest'):
    
    """
    Convert longitude and latitude into a post code, using the street name to narrow 
    the possible post codes down. Two methods possible, discussed below.
    
    Parameters
    ----------
    - latitude (float): latitude of the Zoopla property
    - longitude (float): longitude of the Zoopla property
    - street_name (string): the street name for the Zoopla property
    - method (string) default='closest': algorithm for choosing the postcode
       - closest: uses Euclidean distance to calculate the
         nearest longitude and latitude in the mapping file to the Zoopla property and gets the postcode
       - frequency: gets the most common postcode for the property's street name
    
    Returns
    -------
    - postcode (string): postcode closely mapping the Zoopla property
    """
    
    # get all possible postcodes for the street name
    possible_postcodes = list(epc_df[epc_df['Street'].str.lower()==street_name.lower()]['POSTCODE'])
    
    if method == 'frequency':
        
        final_postcode = get_most_common_postcode(possible_postcodes)
        
        # if there is no most common postcode in the EPC data, use the 'closest' algorithm instead
        if not final_postcode:
            method = 'closest'
        
    if method == 'closest':
        
        possible_postcodes = set(possible_postcodes)
        
        # if set is empty, try all possible postcodes and find nearest one
        if not possible_postcodes:
            final_postcode = get_closest_postcode(latitude, longitude, mapping_df)
        
        # if set is filled, limit search to the postcodes in the set
        else:

            # get mapping dataframe of just the postcodes in the above set
            mapping_df_temp = mapping_df[mapping_df['Postcode'].isin(possible_postcodes)]
            
            # where mapping dataframe filled, otherwise, choose the postcode from the set 
            # whose location is closest to the property longitude and latitude
            if not mapping_df_temp.empty:
                final_postcode = get_closest_postcode(latitude, longitude, mapping_df_temp)
            
            # if the postcodes are not found in the mapping file (likely due to
            # that postcode falling outside the geo boundary of the mapping file),
            # set the postcode to the first element in the possible_postcodes set 
            else:
                final_postcode = sorted(possible_postcodes)[0]
            

    return final_postcode
  

In [11]:
epc_filename_nuneaton = 'epcs_nuneaton.csv'
epc_filename_hinckley = 'epcs_hinckley.csv'

epc_df_nuneaton = pd.read_csv(os.path.join(DATA_FOLDER, epc_filename_nuneaton), dtype=str)
epc_df_hinckley = pd.read_csv(os.path.join(DATA_FOLDER, epc_filename_hinckley), dtype=str)
epc_df = epc_df_nuneaton.append(epc_df_hinckley).drop_duplicates()

In [12]:
print(epc_df.shape)
display(epc_df.head())

(91212, 92)


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,LOCAL_AUTHORITY,CONSTITUENCY,COUNTY,LODGEMENT_DATE,TRANSACTION_TYPE,ENVIRONMENT_IMPACT_CURRENT,ENVIRONMENT_IMPACT_POTENTIAL,ENERGY_CONSUMPTION_CURRENT,ENERGY_CONSUMPTION_POTENTIAL,CO2_EMISSIONS_CURRENT,CO2_EMISS_CURR_PER_FLOOR_AREA,CO2_EMISSIONS_POTENTIAL,LIGHTING_COST_CURRENT,LIGHTING_COST_POTENTIAL,HEATING_COST_CURRENT,HEATING_COST_POTENTIAL,HOT_WATER_COST_CURRENT,HOT_WATER_COST_POTENTIAL,TOTAL_FLOOR_AREA,ENERGY_TARIFF,MAINS_GAS_FLAG,FLOOR_LEVEL,FLAT_TOP_STOREY,FLAT_STOREY_COUNT,MAIN_HEATING_CONTROLS,MULTI_GLAZE_PROPORTION,GLAZED_TYPE,GLAZED_AREA,EXTENSION_COUNT,NUMBER_HABITABLE_ROOMS,NUMBER_HEATED_ROOMS,LOW_ENERGY_LIGHTING,NUMBER_OPEN_FIREPLACES,HOTWATER_DESCRIPTION,HOT_WATER_ENERGY_EFF,HOT_WATER_ENV_EFF,FLOOR_DESCRIPTION,FLOOR_ENERGY_EFF,FLOOR_ENV_EFF,WINDOWS_DESCRIPTION,WINDOWS_ENERGY_EFF,WINDOWS_ENV_EFF,WALLS_DESCRIPTION,WALLS_ENERGY_EFF,WALLS_ENV_EFF,SECONDHEAT_DESCRIPTION,SHEATING_ENERGY_EFF,SHEATING_ENV_EFF,ROOF_DESCRIPTION,ROOF_ENERGY_EFF,ROOF_ENV_EFF,MAINHEAT_DESCRIPTION,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATCONT_DESCRIPTION,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_DESCRIPTION,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,WIND_TURBINE_COUNT,HEAT_LOSS_CORRIDOR,UNHEATED_CORRIDOR_LENGTH,FLOOR_HEIGHT,PHOTO_SUPPLY,SOLAR_WATER_HEATING_FLAG,MECHANICAL_VENTILATION,ADDRESS,LOCAL_AUTHORITY_LABEL,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE
0,1230178709552014110416461495049124,"71, Riversley Road",NaN,NaN,CV11 5QT,1404959278,E,D,45,64,Maisonette,End-Terrace,2014-11-01,E07000219,E14000868,Warwickshire,2014-11-04,none of the above,42,62,384,236,5.1,74,3.1,96,51,989,616,110,110,69.0,dual,Y,1st,Y,NaN,2102,100,double glazing installed before 2002,Normal,1,4,4,11,0,From main system,Good,Good,(other premises below),NaN,NaN,Fully double glazed,Average,Average,"Solid brick, as built, no insulation (assumed)",Very Poor,Very Poor,None,NaN,NaN,"Pitched, no insulation (assumed)",Very Poor,Very Poor,"Boiler and radiators, mains gas",Good,Good,"Programmer, no room thermostat",Very Poor,Very Poor,Low energy lighting in 11% of fixed outlets,Poor,Poor,mains gas (not community),0,no corridor,NaN,NaN,0.0,NaN,natural,"71, Riversley Road",Nuneaton and Bedworth,Nuneaton,NUNEATON,England and Wales: 1900-1929,2014-11-04 16:46:14,owner-occupied,9,1,100070163319,Address Matched
1,868018689262012122112363153798182,"23, Arden Road",Bulkington,NaN,CV12 9JJ,8533893078,D,B,59,85,House,Detached,2012-12-21,E07000219,E14000905,Warwickshire,2012-12-21,FiT application,55,85,251,80,4.6,48,1.5,64,64,795,441,86,63,96.0,Single,Y,NODATA!,NaN,NaN,2106,100,double glazing installed during or after 2002,Normal,1,7,7,75,0,From main system,Good,Good,"Solid, no insulation (assumed)",NaN,NaN,Fully double glazed,Good,Good,"Solid brick, as built, no insulation (assumed)",Very Poor,Very Poor,"Room heaters, mains gas",NaN,NaN,"Pitched, 150 mm loft insulation",Good,Good,"Boiler and radiators, mains gas",Good,Good,"Programmer, room thermostat and TRVs",Good,Good,Low energy lighting in 75% of fixed outlets,Very Good,Very Good,mains gas (not community),0,NO DATA!,NaN,NaN,0.0,NaN,natural,"23, Arden Road, Bulkington",Nuneaton and Bedworth,Rugby,BEDWORTH,England and Wales: 1900-1929,2012-12-21 12:36:31,owner-occupied,12,9,100070135974,Address Matched
2,1008849941732013092016263284978300,"249, Lutterworth Road",NaN,NaN,CV11 6PU,4044393178,E,C,50,76,Bungalow,Detached,2013-09-16,E07000219,E14000868,Warwickshire,2013-09-20,marketed sale,48,75,273,126,7.1,52,3.3,112,73,1292,857,172,84,136.0,dual,Y,NODATA!,NaN,NaN,2106,95,double glazing installed before 2002,Normal,1,6,6,47,1,From main system,Average,Average,"Solid, no insulation (assumed)",NaN,NaN,Mostly double gla

In [13]:
# get street name from addresses
epc_df['Street'] = epc_df[['ADDRESS1', 'ADDRESS2']].apply(lambda x: get_street_name(x[0], x[1]), axis=1)

In [14]:
# try both 'closest' and 'frequency' method to get postcode
zoopla_df['postcode_test_2'] = zoopla_df[['latitude', 'longitude', 'street_name']].apply(lambda x: get_postcode(x[0], x[1], x[2], method='closest'), axis=1)
zoopla_df['postcode_test_3'] = zoopla_df[['latitude', 'longitude', 'street_name']].apply(lambda x: get_postcode(x[0], x[1], x[2], method='frequency'), axis=1)

In [15]:
zoopla_df[['latitude', 'longitude', 'postcode_test_1', 'postcode_test_2', 'postcode_test_3']].head(30)

,latitude,longitude,postcode_test_1,postcode_test_2,postcode_test_3
0,52.520170,-1.455287,CV11 4FS,CV11 4FS,CV11 4FS
1,52.520170,-1.455287,CV11 4FS,CV11 4FS,CV11 4FS
2,52.520170,-1.455287,CV11 4FS,CV11 4FS,CV11 4FS
3,52.520170,-1.455287,CV11 4FS,CV11 4FS,CV11 4FS
4,52.543770,-1.463799,CV10 0FH,CV10 0FH,CV10 0FH
5,52.519066,-1.495866,CV10 7RU,CV10 7DE,CV10 7DE
6,52.541720,-1.433190,CV11 7AB,CV11 6BQ,CV11 6BQ
7,52.525670,-1.534454,CV10 9NG,CV10 9NG,CV10 9NQ
8,52.518400,-1.478219,CV11 5XJ,CV11 5XH,CV11 5XH
9,52.513065,-1.493124,CV10 7HA,CV10 7HA,CV10 7HA


From checks on Google, the 'closest' method is most accurate so will be used for this dataset

In [16]:
zoopla_df.drop(columns=['postcode_test_1', 'postcode_test_3'], inplace=True)
zoopla_df = zoopla_df.rename(columns={'postcode_test_2': 'postcode'})

### Get the parish from the postcode

In [17]:
def get_parish(postcode, town):
    
    """
    Get the parish of the Zoopla property from the mapping dataframe (based on postcode and town)
    """
    
    try:
        return mapping_df[mapping_df['Postcode']==postcode].iloc[0]['Parish']
    except IndexError:
        if town in ['Nuneaton', 'Bedworth', 'Atherstone']:
            return "Nuneaton and Bedworth, unparished area"
        elif town in ['Hinckley', 'Bosworth']:
            return "Hinckley and Bosworth, unparished area"
        else:
            return "Other/Unknown"

In [18]:
zoopla_df['post_town'].value_counts()

Nuneaton      580
Hinckley      292
Bedworth        7
Atherstone      5
Derby           1
Name: post_town, dtype: int64

In [19]:
zoopla_df['parish'] = zoopla_df[['postcode', 'post_town']].apply(lambda x: get_parish(x[0], x[1]), axis=1)

In [20]:
zoopla_df['parish'].value_counts()

Nuneaton and Bedworth, unparished area    568
Hinckley and Bosworth, unparished area    280
Burbage                                    15
Stoke Golding                               9
Higham on the Hill                          5
Witherley                                   3
Barlestone                                  1
Earl Shilton                                1
Sheepy                                      1
Nailstone                                   1
Market Bosworth                             1
Name: parish, dtype: int64

Maybe not such a useful feature but keep for now!

### Save to csv file

In [21]:
zoopla_df.head()

,agent_logo,outcode,price_modifier,num_recepts,street_name,first_published_date,agent_address,property_type,floor_plan,details_url,country,num_bathrooms,agent_name,listing_status,listing_id,price,displayable_address,image_url,latitude,longitude,description,post_town,country_code,county,last_published_date,num_bedrooms,category,agent_phone,postcode,parish
0,https://st.zoocdn.com/zoopla_static_agent_logo...,CV11,from,3,"Meadow Green, Watling Street",2023-02-04 05:28:55,"Meadow Green, Watling Street, Nuneaton",Detached house,NaN,https://www.zoopla.co.uk/for-sale/details/6388...,England,0,Taylor Wimpey - Meadow Green,sale,63883197,376500.0,"""The Lanford - Plot 322"" at Windrower Close, N...",https://lid.zoocdn.com/354/255/fd606582b571af7...,52.52017,-1.455287,"Discover this 4 bedroom Lanford home, ideal fo...",Nuneaton,gb,Warwickshire,2023-02-04 05:40:31,4,Residential,024 7511 6265,CV11 4FS,"Nuneaton and Bedworth, unparished area"
1,https://st.zoocdn.com/zoopla_static_agent_logo...,CV11,from,3,"Meadow Green, Watling Street",2023-02-04 05:28:47,"Meadow Green, Watling Street, Nuneaton",Detached house,NaN,https://www.zoopla.co.uk/for-sale/details/6388...,England,0,Taylor Wimpey - Meadow Green,sale,63883200,489950.0,"""The Ransford - Plot 119"" at Windrower Close, ...",https://lid.zoocdn.com/354/255/f5547b1657bfbfd...,52.52017,-1.455287,This four bedroom Ransford home is perfect for...,Nuneaton,gb,Warwickshire,2023-02-04 05:38:17,4,Residential,024 7511 6265,CV11 4FS,"Nuneaton and Bedworth, unparished area"
2,https://st.zoocdn.com/zoopla_static_agent_logo...,CV11,from,2,"Meadow Green, Watling Street",2023-02-04 05:28:47,"Meadow Green, Watling Street, Nuneaton",Detached house,NaN,https://www.zoopla.co.uk/for-sale/details/6388...,England,0,Taylor Wimpey - Meadow Green,sale,63883198,305000.0,"""The Byford - Plot 323"" at Windrower Close, Nu...",https://lid.zoocdn.com/354/255/941aa37a7610247...,52.52017,-1.455287,Find out how our mortgage contribution scheme*...,Nuneaton,gb,Warwickshire,2023-02-04 05:38:28,3,Residential,024 7511 6265,CV11 4FS,"Nuneaton and Bedworth, unparished area"
3,https://st.zoocdn.com/zoopla_static_agent_logo...,CV11,from,2,"Meadow Green, Watling Street",2023-02-04 05:28:47,"Meadow Green, Watling Street, Nuneaton",Detached house,NaN,https://www.zoopla.co.uk/for-sale/details/6388...,England,0,Taylor Wimpey - Meadow Green,sale,63883199,314950.0,"""The Amersham - Plot 373"" at Windrower Close, ...",https://lid.zoocdn.com/354/255/b4096bb0c276201...,52.52017,-1.455287,A delightful three bedroom home with an integr...,Nuneaton,gb,Warwickshire,2023-02-04 05:40:02,3,Residential,024 7511 6265,CV11 4FS,"Nuneaton and Bedworth, unparished area"
4,https://st.zoocdn.com/zoopla_static_agent_logo...,CV10,guide_price,1,Duckpond Lane,2023-02-03 19:18:48,"22 Newdegate Street, Nuneaton",Detached house,NaN,https://www.zoopla.co.uk/for-sale/details/6388...,England,2,Alan Cooper Estates,sale,63881100,300000.0,"Duckpond Lane, Weddington, Nuneaton CV10",https://lid.zoocdn.com/354/255/7649fd019aaf859...,52.54377,-1.463799,Here is a superb double fronted Detached Resid...,Nuneaton,gb,Warwickshire,2023-02-03 19:37:49,3,Residential,024 7513 8435,CV10 0FH,"Nuneaton and Bedworth, unparished area"


In [22]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file = os.path.join(SAVE_FOLDER, 'zoopla_properties_with_postcode.csv')
    
zoopla_df.to_csv(save_file, index=False)